Loading Libraries

In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
import pandas as pd

Creating a spark session

In [2]:
spark = SparkSession.builder.appName("financial_analysis").getOrCreate()

Reading the CSV file

In [3]:
fin_data = spark.read.csv(r"D:\BCG_Financial_Chatbot\task_1\financial_data.csv",inferSchema=True,header=True)

Finding year by year growth for each element

In [4]:
windowover = Window.partitionBy("Organization").orderBy("Year")
fin_cols = ['Total Revenue','Net Income','Total Assets','Total Liabilities','Cash Flow from Operating Activities']
for i in fin_cols:
    fin_data = fin_data.withColumn(f"{i} Growth Percentage",F.round(((F.col(f"{i}") - F.lag(F.col(f"{i}")).over(windowover))/F.lag(F.col(f"{i}")).over(windowover)) * 100,4))

fin_data = fin_data.fillna(0)

In [5]:
fin_data.show()

+------------+----+-------------+----------+------------+-----------------+-----------------------------------+-------------------------------+----------------------------+------------------------------+-----------------------------------+-----------------------------------------------------+
|Organization|Year|Total Revenue|Net Income|Total Assets|Total Liabilities|Cash Flow from Operating Activities|Total Revenue Growth Percentage|Net Income Growth Percentage|Total Assets Growth Percentage|Total Liabilities Growth Percentage|Cash Flow from Operating Activities Growth Percentage|
+------------+----+-------------+----------+------------+-----------------+-----------------------------------+-------------------------------+----------------------------+------------------------------+-----------------------------------+-----------------------------------------------------+
|       Apple|2021|       365817|     94680|      351002|           287912|                             104038|       

Saving the clean data for training

In [ ]:
data = pd.DataFrame(columns=["instruction","output"])
for i in fin_data.head(fin_data.count()):
    for j in list(fin_data.columns[2:]):

        dat = {
            "instruction":f"What is the {j} of {i['Organization']} in {i['Year']}",
            "output":f"The {j} of {i['Organization']} in {i['Year']} is {i[f'{j}']}"
        }
        dat = pd.DataFrame(dat,index=[0])
        data = pd.concat([data,dat])

data = data.reset_index(drop=True)
data.to_csv("bcg_fin_train_data.csv",index=False)

### Year-on-Year Average Growth Rates of Apple, Microsoft and Tesla

In [12]:
fin_data.select("Organization","Total Revenue Growth Percentage","Net Income Growth Percentage","Total Assets Growth Percentage","Total Liabilities Growth Percentage","Cash Flow from Operating Activities Growth Percentage")\
    .groupby(["Organization"])\
    .agg(
        F.mean(F.col("Total Revenue Growth Percentage")).alias("Total Revenue Growth Percentage %"),
        F.mean(F.col("Net Income Growth Percentage")).alias("Net Income Growth Percentage %"),
        F.mean(F.col("Total Assets Growth Percentage")).alias("Total Assets Growth Percentage %"),
        F.mean(F.col("Total Liabilities Growth Percentage")).alias("Total Liabilities Growth Percentage %"),
        F.mean(F.col("Cash Flow from Operating Activities Growth Percentage")).alias("Cash Flow from Operating Activities Growth Percentage %"),
        )\
    .show()
    

+------------+---------------------------------+------------------------------+--------------------------------+-------------------------------------+-------------------------------------------------------+
|Organization|Total Revenue Growth Percentage %|Net Income Growth Percentage %|Total Assets Growth Percentage %|Total Liabilities Growth Percentage %|Cash Flow from Operating Activities Growth Percentage %|
+------------+---------------------------------+------------------------------+--------------------------------+-------------------------------------+-------------------------------------------------------+
|       Apple|               1.6644333333333332|                        0.8658|                          0.1502|                   0.3555999999999999|                                      2.635666666666667|
|   Microsoft|                8.279366666666666|             6.065633333333333|                          7.4085|                               2.3841|                      